In [105]:
from feature.make_dataset import make_data
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')


In [106]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [107]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)

c:\mb_job\project\Competition\Lettuce-Growth-Environment-Prediction\feature\base_dataset.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cum_col][i*24:(i+1)*24] = cumsum
c:\mb_job\project\Competition\Lettuce-Growth-Environment-Prediction\feature\base_dataset.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cum_col][i*24:(i+1)*24] = cumsum


In [109]:
train_1 = pd.read_csv('./data/preprocessing/train_quantile0.25.csv')
test_1 = pd.read_csv('./data/preprocessing/test_quantile0.25.csv')
train_3 = pd.read_csv('./data/preprocessing/train_quantile0.75.csv')
test_3 = pd.read_csv('./data/preprocessing/test_quantile0.75.csv')
train_std = pd.read_csv('./data/preprocessing/train_std.csv')
test_std = pd.read_csv('./data/preprocessing/test_std.csv')
train_mid = pd.read_csv('./data/preprocessing/train_median.csv')
test_mid = pd.read_csv('./data/preprocessing/test_median.csv')

In [110]:
train_3.columns = [col+' 3' for col in train_3.columns]
test_3.columns = [col+' 3' for col in test_3.columns]
train_1.columns = [col+' 1' for col in train_1.columns]
test_1.columns = [col+' 1' for col in test_1.columns]
train_std.columns = [col+' std' for col in train_std.columns]
test_std.columns = [col+' std' for col in test_std.columns]
train_mid.columns = [col+' mid' for col in train_mid.columns]
test_mid.columns = [col+' mid' for col in test_mid.columns]

In [111]:
train = train.reset_index()
test  = test.reset_index()
train_3 = train_3.reset_index()
test_3 = test_3.reset_index()
train_1 = train_1.reset_index()
test_1 = test_1.reset_index()
train_std = train_std.reset_index()
test_std = test_std.reset_index()
train_mid = train_mid.reset_index()
test_mid = test_mid.reset_index()


In [112]:
train = pd.concat([train, train_1, train_3, train_std, train_mid], axis=1)
test = pd.concat([test, test_1, test_3, test_std, test_mid], axis=1)

In [113]:
train = train.drop(['index'], axis=1)
test = test.drop(['index'], axis=1)

In [119]:
train = pd.concat([train, p_train], axis=1)
test = pd.concat([test, p_test], axis=1)

In [121]:
def model_train(train, test, model):
    '''
    train shape = (784, 18), test shape = (140, 18)
    model => sklearn api model
    '''

    X = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    y = train['predicted_weight_g']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=113, shuffle=True)
    
    x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    rmse = mean_squared_error(y_val, y_val_pred)**0.5
    print(f"validation rmse: {rmse}")
    
    x_train = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    y_train = train['predicted_weight_g']
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    x_test['predicted_weight_g'] = y_pred
    submit = x_test[['DAT', 'predicted_weight_g']]
    submit['DAT'] = submit['DAT']+1
    all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
    for idx, test_path in enumerate(all_target_list):
        submit_df = pd.read_csv(test_path)
        submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
        submit_df.to_csv(test_path, index=False)

In [124]:
xgb = XGBRegressor()
cat = CatBoostRegressor(verbose=100, early_stopping_rounds=50, task_type="GPU")
lgbm = LGBMRegressor()
linear = LinearRegression()

model_train(train, test, cat)

Learning rate set to 0.037333
0:	learn: 40.7703422	total: 9.08ms	remaining: 9.07s
100:	learn: 8.3556613	total: 936ms	remaining: 8.33s
200:	learn: 5.8776129	total: 1.85s	remaining: 7.35s
300:	learn: 5.0483855	total: 2.76s	remaining: 6.42s
400:	learn: 4.7099678	total: 3.65s	remaining: 5.46s
500:	learn: 4.4108802	total: 4.55s	remaining: 4.53s
600:	learn: 4.0666769	total: 5.46s	remaining: 3.63s
700:	learn: 3.6751749	total: 6.38s	remaining: 2.72s
800:	learn: 3.3572233	total: 7.29s	remaining: 1.81s
900:	learn: 3.2345092	total: 8.16s	remaining: 896ms
999:	learn: 3.1544748	total: 9.07s	remaining: 0us
validation rmse: 9.166773932467919
Learning rate set to 0.039126
0:	learn: 40.3695066	total: 9.47ms	remaining: 9.46s
100:	learn: 8.0300737	total: 916ms	remaining: 8.15s
200:	learn: 6.0080861	total: 1.8s	remaining: 7.15s
300:	learn: 5.1377865	total: 2.72s	remaining: 6.32s
400:	learn: 4.6357647	total: 3.59s	remaining: 5.36s
500:	learn: 4.3169911	total: 4.49s	remaining: 4.47s
600:	learn: 4.0623519	to

C:\Users\jkpark\AppData\Local\Temp\ipykernel_31256\3184216771.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['DAT'] = submit['DAT']+1
